In [ ]:
import os
import time
import glob
import numpy as np
import cv2
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from IPython.display import clear_output  # Import for clearing output

# Load the saved model
model_fruit = load_model('fruit_classification_model1.h5')

# Define categories
categories = ['freshapples', 'freshbanana', 'freshoranges', 
              'rottenapples', 'rottenbanana', 'rottenoranges']

def load_image_for_prediction(img_path, target_size=(100, 100)):
    """Load and preprocess the image for prediction."""
    img = image.load_img(img_path, target_size=target_size)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = img_array / 255.0  # Normalize
    return img_array

def predict_fruit_category(model, img_path):
    """Make a prediction on the given image."""
    img_array = load_image_for_prediction(img_path)
    prediction = model.predict(img_array)
    predicted_label_index = np.argmax(prediction)
    predicted_label = categories[predicted_label_index]
    predicted_probability = prediction[0][predicted_label_index]
    return predicted_label, predicted_probability

# Directory to monitor
image_dir = "captured_images"

# Start monitoring the directory
print("Monitoring the directory for new images...")
while True:
    # Get all image files in the directory
    image_files = glob.glob(os.path.join(image_dir, "image-*.jpg"))

    if image_files:
        for img_path in image_files:
            print(f"Processing image: {img_path}")
            
            # Make a prediction
            result, probability = predict_fruit_category(model_fruit, img_path)

            # Debugging output
            print(f"Predicted label: {result}, Probability: {probability:.2f}")

            # Print result if confidence is above a certain threshold
            if probability > 0.5:
                print(f"The fruit is: {result} with confidence: {probability:.2f}")
                
                # Display the captured image with Matplotlib
                img = cv2.imread(img_path)
                plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
                plt.title(f'Prediction: {result}, Confidence: {probability:.2f}')
                plt.axis('off')
                plt.show()
            else:
                print("Confidence too low for prediction.")

            # Delete the image after processing
            # os.remove(img_path)
            # print(f"Deleted image: {img_path}")

            # Clear the output and wait before processing the next image
            clear_output(wait=True)  # Clear output in Jupyter Notebook/VS Code
            time.sleep(3)  # Wait for 3 seconds before the next prediction

    else:
        print("No new images found. Checking again...")
        time.sleep(1)  # Wait a bit before checking again
